In [ ]:
%matplotlib inline

# IDF-based MDO on the Sobieski SSBJ test case.


In [ ]:
from __future__ import annotations

from gemseo import create_discipline
from gemseo import create_scenario
from gemseo import generate_n2_plot
from gemseo.problems.mdo.sobieski.core.design_space import SobieskiDesignSpace
from gemseo.settings.opt import SLSQP_Settings

## Instantiate the  disciplines
First, we instantiate the four disciplines of the use case:
[SobieskiPropulsion][gemseo.problems.mdo.sobieski.disciplines.SobieskiPropulsion],
[SobieskiAerodynamics][gemseo.problems.mdo.sobieski.disciplines.SobieskiAerodynamics],
[SobieskiMission][gemseo.problems.mdo.sobieski.disciplines.SobieskiMission]
and [SobieskiStructure][gemseo.problems.mdo.sobieski.disciplines.SobieskiStructure].



In [ ]:
disciplines = create_discipline([
    "SobieskiPropulsion",
    "SobieskiAerodynamics",
    "SobieskiMission",
    "SobieskiStructure",
])

We can quickly access the most relevant information of any discipline (name, inputs,
and outputs) with Python's `print()` function. Moreover, we can get the default
input values of a discipline with the attribute [Discipline.default_input_data][gemseo.core.discipline.discipline.Discipline.default_input_data].



In [ ]:
for discipline in disciplines:
    for discipline in disciplines:
        print(discipline)
        print(f"Default inputs: {discipline.default_input_data}")

You may also be interested in plotting the couplings of your disciplines.
A quick way of getting this information is the API function
[generate_n2_plot()][gemseo.generate_n2_plot]. A much more detailed explanation of coupling
visualization is available [here][coupling-visualization].



In [ ]:
generate_n2_plot(disciplines, save=False, show=True)

## Build, execute and post-process the scenario
Then, we build the scenario which links the disciplines
with the formulation and the optimization algorithm. Here, we use the
[IDF][gemseo.formulations.idf.IDF] formulation. We tell the scenario to minimize -y_4 instead of
minimizing y_4 (range), which is the default option.

### Instantiate the scenario



In [ ]:
design_space = SobieskiDesignSpace()
design_space

In [ ]:
scenario = create_scenario(
    disciplines,
    "y_4",
    design_space,
    maximize_objective=True,
    formulation_name="IDF",
)

### Set the design constraints



In [ ]:
for c_name in ["g_1", "g_2", "g_3"]:
    scenario.add_constraint(c_name, constraint_type="ineq")

### Visualize the XDSM
Generate the XDSM file on the fly:

- `log_workflow_status=True` will log the status of the workflow  in the console,
- `save_html` (default `True`) will generate a self-contained HTML file,
  that can be automatically opened using `show_html=True`.



In [ ]:
scenario.xdsmize(save_html=False)

### Define the algorithm inputs
We set the maximum number of iterations, the optimizer
and the optimizer settings



In [ ]:
slsqp_settings = SLSQP_Settings(
    max_iter=20,
    ftol_rel=1e-10,
    ineq_tolerance=1e-3,
    eq_tolerance=1e-3,
    normalize_design_space=True,
)

### Execute the scenario



In [ ]:
scenario.execute(slsqp_settings)

### Save the optimization history
We can save the whole optimization problem and its history for further post
processing:



In [ ]:
scenario.save_optimization_history("idf_history.h5", file_format="hdf5")

We can also save only calls to functions and design variables history:



In [ ]:
scenario.save_optimization_history("idf_history.xml", file_format="ggobi")

### Print optimization metrics



In [ ]:
scenario.print_execution_metrics()

### Plot the optimization history view



In [ ]:
scenario.post_process(post_name="OptHistoryView", save=False, show=True)

### Plot the quadratic approximation of the objective



In [ ]:
scenario.post_process(post_name="QuadApprox", function="-y_4", save=False, show=True)